In [83]:
# Import necessary modules from PyTorch
import torchvision
import torch
from torchvision import transforms, datasets

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Preprosessing Data

In [84]:
# Data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize the image
    transforms.RandomHorizontalFlip(),   # Randomly flip the image horizontally
    transforms.RandomRotation(10),       # Randomly rotate the image by up to 10 degrees
    transforms.ColorJitter(0.2, 0.2, 0.2),  # Randomly adjust brightness, contrast, and saturation
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

val_transform = transforms.Compose([
    transforms.Resize(256),              # Resize the image to 256x256
    transforms.CenterCrop(224),          # Crop the center of the image to 224x224
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Define dataset root directory
data_dir = 'dataset_flower102/'

In [85]:
# Apply transformations to the dataset during data loading
train_dataset = datasets.Flowers102(root=data_dir, split='train', transform=train_transform, download=True)
valid_dataset = datasets.Flowers102(root=data_dir, split='val', transform=val_transform, download=True)
# test_dataset = datasets.Flowers102(root=data_dir, split='test', transform=data_transforms, download=True)

In [86]:
# Create data loaders
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=12, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## CNN Model Architecture

In [87]:
# Import necessary modules for Neural Network
import torch.nn as nn
import torch.optim as optim

In [88]:
# Define custom Convolution Neural Network
# Simple CNN architecture with two convolutional layers followed by max pooling, two fully connected layers, and a dropout layer for regularization.
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

        # Max pooling layer: down-sample an image by applying max filer to subregion
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 56 * 56, 512)  # Adjust input size based on image dimensions
        self.fc2 = nn.Linear(512, num_classes)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)

    
    # Defines the forward pass of the network, where input data x is passed through each layer sequentially.
    def forward(self, x):
        # Convolutional layers with ReLU activation and max pooling
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        # Flatten the output from convolutional layers
        x = x.view(-1, 32 * 56 * 56)  # Adjust size based on image dimensions
        # Fully connected layers with dropout
        x = self.dropout(nn.ReLU()(self.fc1(x)))
        x = self.fc2(x)
        return x
    


    # def __init__(self, num_classes):
    #     super(CNN, self).__init__()
    #     # Convolutional layers
    #     self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
    #     self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    #     self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    #     # Max pooling layers
    #     self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
    #     # Fully connected layers
    #     self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Adjust input size based on image dimensions
    #     self.fc2 = nn.Linear(512, num_classes)
    #     # Dropout layer to prevent overfitting
    #     self.dropout = nn.Dropout(p=0.5)
    
    # def forward(self, x):
    #     # Convolutional layers with ReLU activation and max pooling
    #     x = self.pool(nn.ReLU()(self.conv1(x)))
    #     x = self.pool(nn.ReLU()(self.conv2(x)))
    #     x = self.pool(nn.ReLU()(self.conv3(x)))
    #     # Flatten the output from convolutional layers
    #     x = x.view(-1, 128 * 28 * 28)  # Adjust size based on image dimensions
    #     # Fully connected layers with dropout
    #     x = self.dropout(nn.ReLU()(self.fc1(x)))
    #     x = self.fc2(x)
    #     return x


In [89]:
# Create an instance of the CNN model
model = CNN(num_classes=102).to(device)  # 102 classes for Flowers102 dataset

## CNN Model Training

### Set Training Parameters

In [94]:
# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Number of training epochs
num_epochs = 200

### Model Training and Validation

In [98]:
# Load the best trained model along with training and validation loss values
checkpoint = torch.load('best_trained_model.pth')
best_model_state = checkpoint['model_state_dict']
train_loss = checkpoint['train_loss']
val_loss = checkpoint['val_loss']

# Set the model state and loss values before resuming training (Need to comment out the train_loss and val_loss)
model.load_state_dict(best_model_state)

best_val_loss = float('inf')  # Initialize the best validation loss with a large value
patience = 50  # Number of epochs to wait before stopping if validation loss doesn't improve
best_model_state = None

for epoch in range(num_epochs):
    
    # Train the model
    model.train() # Set the model to training mode
    # train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device) # Move images and labels to GPU
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = model.forward(images) # Forward pass
        loss = loss_fn(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass
        optimizer.step() # Optimize

        train_loss += loss.item() * images.size(0) #  scalar value of the loss tensor for the current batch * the batch size to account for the loss per sample in the batch
        _, predicted = outputs.max(1) # Returns a tuple containing the maximum value along the specified dimension (class probabilities for each sample in the batch) and index of the max value
        total += labels.size(0) # Accumulates the total number of sample seen during training
        correct += predicted.eq(labels).sum().item() # Accumulates the total number of correct predictions over all batches.
    
    # Calculate training loss and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100.0 * correct / total

    # Print training loss and accuracy
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')



    # Validate the model
    model.eval()  # Set the model to evaluation mode
    # val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device) # Move images and labels to GPU
            outputs = model.forward(images)  # Forward pass
            loss = loss_fn(outputs, labels)  # Calculate the loss

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Calculate validation loss and accuracy
    val_loss = val_loss / len(valid_loader.dataset)
    val_accuracy = 100.0 * correct / total

    # Print validation loss and accuracy
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Check if validation loss has improved
    if val_loss < best_val_loss:
        print("Creating new checkpoint for best model...")
        best_val_loss = val_loss
        patience = 50  # Reset patience if validation loss improves
        best_model_state = model.state_dict()  # Save the best model state
    # else:
    #     patience -= 1
    #     if patience == 0:
    #         print("Early stopping...")
    #         break

# Save the best trained model along with training and validation loss values
torch.save({
    'model_state_dict': best_model_state,
    'train_loss': train_loss,
    'val_loss': val_loss
}, 'best_trained_model.pth')

Epoch 1/200, Training Loss: 1.6983, Training Accuracy: 49.90%
Epoch 1/200, Validation Loss: 2.9001, Validation Accuracy: 43.53%
Epoch 2/200, Training Loss: 1.6783, Training Accuracy: 51.96%
Epoch 2/200, Validation Loss: 2.9554, Validation Accuracy: 42.06%
Epoch 3/200, Training Loss: 1.6508, Training Accuracy: 55.00%
Epoch 3/200, Validation Loss: 3.0144, Validation Accuracy: 41.96%
Epoch 4/200, Training Loss: 1.6884, Training Accuracy: 52.06%
Epoch 4/200, Validation Loss: 2.8546, Validation Accuracy: 42.75%
Epoch 5/200, Training Loss: 1.6742, Training Accuracy: 53.82%
Epoch 5/200, Validation Loss: 2.9239, Validation Accuracy: 42.25%
Epoch 6/200, Training Loss: 1.8098, Training Accuracy: 50.29%
Epoch 6/200, Validation Loss: 3.0392, Validation Accuracy: 41.67%
Epoch 7/200, Training Loss: 1.7551, Training Accuracy: 52.35%
Epoch 7/200, Validation Loss: 2.8046, Validation Accuracy: 41.27%
Epoch 8/200, Training Loss: 1.7158, Training Accuracy: 51.37%
Epoch 8/200, Validation Loss: 2.9825, Vali